<a href="https://colab.research.google.com/github/Theseyh/Big-Data-Framework/blob/main/Rendu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os

os.environ["SPARK_VERSION"] = "spark-3.5.3"
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget  http://apache.osuosl.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!echo $SPARK_VERSION-bin-hadoop3.tgz
!rm $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 129 kB in 1s (86.4 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
--2024-11-26 13:17:34--  htt

### Set Environment Variables
Set the locations where Spark and Java are installed.

In [19]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark/"
os.environ["DRIVE_DATA"] = "/content/gdrive/My Drive/Big Data Framework/data"

!rm /content/spark
!ln -s /content/$SPARK_VERSION-bin-hadoop3 /content/spark
!export PATH=$PATH:$SPARK_HOME/bin:$SPARK_HOME/sbin
!echo $SPARK_HOME
!env |grep  "DRIVE_DATA"

/content/spark/
DRIVE_DATA=/content/gdrive/My Drive/Big Data Framework/data


### Start a SparkSession
This will start a local Spark session.

In [20]:
!python -V

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

# Example: shows the PySpark version
print("PySpark version {0}".format(sc.version))

# Example: parallelise an array and show the 2 first elements
sc.parallelize([2, 3, 4, 5, 6]).cache().take(2)

Python 3.10.12
PySpark version 3.5.3


[2, 3]

In [21]:
from pyspark.sql import SparkSession
# We create a SparkSession object (or we retrieve it if it is already created)
spark = SparkSession \
.builder \
.appName("My application") \
.config("spark.some.config.option", "some-value") \
.master("local[4]") \
.getOrCreate()
# We get the SparkContext
sc = spark.sparkContext

## Exercise 3.1: Word count

Count the number of words *per line* in the $DRIVE_DATA/quijote.txt file.

Repeat the exercise but this time counting the number of words *in the whole file*.

In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
!head "$DRIVE_DATA/quijote.txt"

﻿The Project Gutenberg EBook of Don Quijote, by Miguel de Cervantes Saavedra

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.net


Title: Don Quijote



In [24]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Initialize Spark session
spark = SparkSession.builder.appName("WordCount").getOrCreate()

# Define file path
file_path = "/content/gdrive/My Drive/Big Data Framework/data/quijote.txt"

# Load the file into a DataFrame, with one line per row
lines_df = spark.read.text(file_path)

# Count words per line
words_per_line_df = lines_df.withColumn("word_count", F.size(F.split(F.col("value"), " ")))

# Show the word counts per line
words_per_line_df.show(truncate=False)

# Count total words in the whole file
total_words = words_per_line_df.select(F.sum("word_count").alias("total_word_count")).collect()[0]["total_word_count"]

print(f"Total number of words in the file: {total_words}")

# Stop the Spark session
spark.stop()


+---------------------------------------------------------------------------+----------+
|value                                                                      |word_count|
+---------------------------------------------------------------------------+----------+
|The Project Gutenberg EBook of Don Quijote, by Miguel de Cervantes Saavedra|12        |
|                                                                           |1         |
|This eBook is for the use of anyone anywhere at no cost and with           |14        |
|almost no restrictions whatsoever.  You may copy it, give it away or       |13        |
|re-use it under the terms of the Project Gutenberg License included        |11        |
|with this eBook or online at www.gutenberg.net                             |7         |
|                                                                           |1         |
|                                                                           |1         |
|Title: Don Quijote  

V2


In [25]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Initialize Spark session
spark = SparkSession.builder.appName("WordCount").getOrCreate()

# Define file path
file_path = "/content/gdrive/My Drive/Big Data Framework/data/quijote.txt"

# Load the file into a DataFrame, with one line per row
lines_df = spark.read.text(file_path)

# Filter out empty lines (lines that are empty or only contain whitespace)
non_empty_lines_df = lines_df.filter(F.trim(F.col("value")) != "")

# Count words per line
words_per_line_df = non_empty_lines_df.withColumn("word_count", F.size(F.split(F.col("value"), " ")))

# Show the word counts per line
words_per_line_df.show(truncate=False)

# Count total words in the whole file
total_words = words_per_line_df.select(F.sum("word_count").alias("total_word_count")).collect()[0]["total_word_count"]

print(f"Total number of words in the file: {total_words}")

# Stop the Spark session
spark.stop()


+---------------------------------------------------------------------------+----------+
|value                                                                      |word_count|
+---------------------------------------------------------------------------+----------+
|The Project Gutenberg EBook of Don Quijote, by Miguel de Cervantes Saavedra|12        |
|This eBook is for the use of anyone anywhere at no cost and with           |14        |
|almost no restrictions whatsoever.  You may copy it, give it away or       |13        |
|re-use it under the terms of the Project Gutenberg License included        |11        |
|with this eBook or online at www.gutenberg.net                             |7         |
|Title: Don Quijote                                                         |3         |
|Author: Miguel de Cervantes Saavedra                                       |5         |
|Posting Date: April 27, 2010 [EBook #2000]                                 |7         |
|Release Date: Decemb

V3

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Initialize Spark session
spark = SparkSession.builder.appName("WordCount").getOrCreate()

# Define file path
file_path = "/content/gdrive/My Drive/Big Data Framework/data/quijote.txt"

# Load the file into a DataFrame, with one line per row
lines_df = spark.read.text(file_path)

# Filter out empty lines (lines that are empty or only contain whitespace)
non_empty_lines_df = lines_df.filter(F.trim(F.col("value")) != "")

# Count words per line
words_per_line_df = non_empty_lines_df.withColumn("word_count", F.size(F.split(F.col("value"), "\s+")))

# Show the word counts per line
words_per_line_df.show(truncate=False)

# Count total words in the whole file
total_words = words_per_line_df.select(F.sum("word_count").alias("total_word_count")).collect()[0]["total_word_count"]

print(f"Total number of words in the file: {total_words}")

# Stop the Spark session
spark.stop()


+---------------------------------------------------------------------------+----------+
|value                                                                      |word_count|
+---------------------------------------------------------------------------+----------+
|The Project Gutenberg EBook of Don Quijote, by Miguel de Cervantes Saavedra|12        |
|This eBook is for the use of anyone anywhere at no cost and with           |14        |
|almost no restrictions whatsoever.  You may copy it, give it away or       |12        |
|re-use it under the terms of the Project Gutenberg License included        |11        |
|with this eBook or online at www.gutenberg.net                             |7         |
|Title: Don Quijote                                                         |3         |
|Author: Miguel de Cervantes Saavedra                                       |5         |
|Posting Date: April 27, 2010 [EBook #2000]                                 |7         |
|Release Date: Decemb

## Exercise 4.1: Pi Estimation

Using the Monte Carlo method, estimate the value of Pi. Use the random() method from the random class.

In [27]:
from pyspark.sql import SparkSession
import random

# Initialize Spark session
spark = SparkSession.builder.appName("MonteCarloPi").getOrCreate()

# Number of random points to generate
NUM_SAMPLES = 10_000_000

# Function to generate a random point and check if it's inside the unit circle
def is_point_inside_unit_circle(_):
    x = random.uniform(-1, 1)
    y = random.uniform(-1, 1)
    return 1 if x**2 + y**2 <= 1 else 0

# Parallelize the calculation across Spark workers
rdd = spark.sparkContext.parallelize(range(NUM_SAMPLES))
points_inside_circle = rdd.map(is_point_inside_unit_circle).reduce(lambda a, b: a + b)

# Approximate π
pi_estimate = (4 * points_inside_circle) / NUM_SAMPLES

print(f"Estimated value of π: {pi_estimate}")

# Stop the Spark session
spark.stop()


Estimated value of π: 3.1417692


## Exercise 4.2: Inspect a log file

Upload the file /var/log/syslog from your computer to this notebook. Then, select only the "bad lines": WARNING and ERROR messages.

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Initialize Spark session
spark = SparkSession.builder.appName("SyslogFilter").getOrCreate()

# Load the syslog file into a DataFrame
#file_path = "/content/syslog"  # Adjust path if necessary after upload
#syslog_df = spark.read.text(file_path)

syslog_df = spark.read.text(os.environ["DRIVE_DATA"] + "/syslog")

# Filter lines containing WARNING or ERROR
bad_lines_df = syslog_df.filter(
    F.col("value").contains("WARNING") | F.col("value").contains("ERROR")
)

# Show the filtered "bad lines"
bad_lines_df.show(truncate=False)

# Stop the Spark session
spark.stop()


+-------------------------------------------------------------------------------------------------+
|value                                                                                            |
+-------------------------------------------------------------------------------------------------+
|Nov 26 00:01:13 student-laptop ovpn-cytech.students[1260]: WARNING: Your certificate has expired!|
|Nov 26 00:12:13 student-laptop ovpn-cytech.students[1260]: WARNING: Your certificate has expired!|
|Nov 26 00:23:14 student-laptop ovpn-cytech.students[1260]: WARNING: Your certificate has expired!|
|Nov 26 00:34:14 student-laptop ovpn-cytech.students[1260]: WARNING: Your certificate has expired!|
|Nov 26 00:45:14 student-laptop ovpn-cytech.students[1260]: WARNING: Your certificate has expired!|
|Nov 26 00:56:14 student-laptop ovpn-cytech.students[1260]: WARNING: Your certificate has expired!|
|Nov 26 01:07:15 student-laptop ovpn-cytech.students[1260]: WARNING: Your certificate has expired!|
